# Génération de partition de musique : réseau LSTM

In [95]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from typing import Dict, List, Optional, Sequence, Tuple
import collections
import numpy as np
import pandas as pd
import pretty_midi
import music21

import json
import os
import sys
sys.path.append('C:/Users/melan/Documents/M2_S10 IAFA/CHEF D\'OEUVRE')
import extract_data as ed
import visualization as vz

### Préparation du dataset

In [2]:
# Creating the required variables
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [3]:
with open("data/data.json",'r') as file:
        data = json.load(file)
                
filenames = [data[i]['title'] for i in range(len(data))]
print('Number of files:', len(filenames))

Number of files: 171


In [24]:
def extract_notes_from(jsonfile,score,part):
   
    title,part,key,notes_duration = ed.extract_seq_from(jsonfile,score,part)
    
    notes = {'name':[],'duration':[]}
    
    for nd in notes_duration:
        
        splitted = nd.split('-')
        if len(splitted) == 2:
            note, duration = pretty_midi.note_name_to_number(splitted[0]), float(splitted[1])
        else:
            note = pretty_midi.note_name_to_number("b".join(substring for substring in splitted[:-1]))
            duration = float(splitted[-1])
        notes['name'].append(note)
        notes['duration'].append(duration)

    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

sample_file = filenames[0]
print(sample_file)

raw_notes = extract_notes_from("data/data.json",'A Ginga do Man\u00e9','A')
raw_notes.head()

A Ginga do Mané


,name,duration
0,88,0.25
1,79,0.25
2,80,0.25
3,81,0.25
4,82,0.25


### Création du dataset d'entraînement

In [25]:
num_files = len(filenames)

all_notes_A = []
all_notes_B = []
all_notes_C = []
all_notes_D = []

for f in filenames:
    
    notesA = extract_notes_from("data/data.json",f,'A')
    all_notes_A.append(notesA)

all_notes_A = pd.concat(all_notes_A)

n_notes_A = len(all_notes_A)
print('Number of notes parsed for part A:', n_notes_A)

key_order = ['name', 'duration']
train_notes_A = np.stack([all_notes_A[key] for key in key_order], axis=1)

notes_ds_A = tf.data.Dataset.from_tensor_slices(train_notes_A)

notes_ds_A.element_spec

Number of notes parsed for part A: 18185


TensorSpec(shape=(2,), dtype=tf.float64, name=None)

In [26]:
def create_sequences(dataset, seq_length):
    """Returns TF Dataset of sequence and label examples."""
    
    seq_length = seq_length+1

    # Take 1 extra for the labels
    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

    # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    # Split the labels
    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]
        labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

        return inputs,labels

    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    
seq_length = 50
seq_ds_A = create_sequences(notes_ds_A, seq_length)

In [28]:
batch_size = 64
buffer_size = n_notes_A - seq_length  # the number of items in the dataset
train_ds_A = (seq_ds_A
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

### Développement du modèle

In [29]:
def mse_with_positive_pressure(y_true, y_pred):
    mse = (y_true - y_pred) ** 2
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

In [30]:
# Developing the model

input_shape = (seq_length, 2)
learning_rate = 0.005

inputs = Input(input_shape)
x = LSTM(128)(inputs)

outputs = {'name': Dense(128, name='name')(x),
           'duration': Dense(1, name='duration')(x),
          }

modelA = Model(inputs, outputs)

loss = {'name': SparseCategoricalCrossentropy(from_logits=True),
        'duration': mse_with_positive_pressure,
       }

optimizer = Adam(learning_rate=learning_rate)

modelA.compile(loss=loss, optimizer=optimizer)

modelA.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 50, 2)]              0         []                            
                                                                                                  
 lstm_2 (LSTM)               (None, 128)                  67072     ['input_3[0][0]']             
                                                                                                  
 duration (Dense)            (None, 1)                    129       ['lstm_2[0][0]']              
                                                                                                  
 name (Dense)                (None, 128)                  16512     ['lstm_2[0][0]']              
                                                                                            

In [31]:
# Creating the necessary callbacks

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./training_checkpoints/ckpt_{epoch}', save_weights_only=True),
             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, 
                                              verbose=1, restore_best_weights=True),]

In [33]:
# Compiling and fitting the model

modelA.compile(loss = loss, 
              loss_weights = {'name': 0.05, 'duration':1.0,},
              optimizer = optimizer)

epochs = 50

history = modelA.fit(train_ds_A, 
                    epochs=epochs, 
                    callbacks=callbacks,)

Epoch 1/50
283/283 [==============================] - 21s 51ms/step - loss: 0.2059 - duration_loss: 0.0595 - name_loss: 2.9280
Epoch 2/50
283/283 [==============================] - 14s 50ms/step - loss: 0.1967 - duration_loss: 0.0592 - name_loss: 2.7500
Epoch 3/50
283/283 [==============================] - 14s 49ms/step - loss: 0.1933 - duration_loss: 0.0587 - name_loss: 2.6922
Epoch 4/50
283/283 [==============================] - 14s 51ms/step - loss: 0.1913 - duration_loss: 0.0581 - name_loss: 2.6644
Epoch 5/50
283/283 [==============================] - 15s 52ms/step - loss: 0.1901 - duration_loss: 0.0577 - name_loss: 2.6473
Epoch 6/50
283/283 [==============================] - 15s 52ms/step - loss: 0.1891 - duration_loss: 0.0574 - name_loss: 2.6352
Epoch 7/50
283/283 [==============================] - 14s 51ms/step - loss: 0.1884 - duration_loss: 0.0570 - name_loss: 2.6277
Epoch 8/50
283/283 [==============================] - 14s 51ms/step - loss: 0.1877 - duration_loss: 0.0565 - na

### Génération de notes

In [49]:
def predict_next_note(notes, model, temperature = 1.0):
    """Generates a note IDs using a trained sequence model."""

    assert temperature > 0

    # Add batch dimension
    inputs = tf.expand_dims(notes, 0)

    predictions = model.predict(inputs)
    name_logits = predictions['name']
    duration = predictions['duration']

    name_logits /= temperature
    name = tf.random.categorical(name_logits, num_samples=1)
    name = tf.squeeze(name, axis=-1)
    duration = tf.squeeze(duration, axis=-1)

    # `duration` values should be non-negative
    #duration = tf.maximum(0, duration)
    quarter_duration = [0.0625,0.125,0.25,0.5,1.0,2.0]

    duration = min(quarter_duration, key=lambda x:abs(x-duration))

    return int(name), float(duration)

In [79]:
def generated_notes_to_dict(jsonfile,score,part,temperature=2.0,num_predictions=100):
    
    raw_notes = extract_notes_from(jsonfile,score,part)
    sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)
    input_notes = sample_notes[:seq_length]
    
    original_notes = [[pretty_midi.note_number_to_name(name),str(duration)] for name,duration in sample_notes]
    original_notes = ["-".join(substring for substring in note) for note in original_notes]
    initial_notes = original_notes[:seq_length]
    
    generated_notes = []
    
    for _ in range(num_predictions):
        
        if part == 'A':
            name, duration = predict_next_note(input_notes, modelA, temperature)
            
        new_note = [pretty_midi.note_number_to_name(name),str(duration)]
        new_note = "-".join(substring for substring in new_note)
        generated_notes.append(new_note)
        
        input_note = (name, duration)
        input_notes = np.delete(input_notes, 0, axis=0)
        input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
    
    return {'Title':score,'Part':part,'Key':"F major",'Start_sequence':initial_notes,'Generated':generated_notes,'Original':original_notes}

In [88]:
generated_scores = []

for f in filenames[:10]:
    
    gen_notes = generated_notes_to_dict("data/data.json",f,'A')
    generated_scores.append(gen_notes)
    
with open('Generated/LSTM.json','w') as file:
    json.dump(generated_scores,file) 

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 40ms/step


In [100]:
music21.environment.set('musescoreDirectPNGPath',str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
music21.environment.set('musicxmlPath', str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
vz.show_all_generated("Generated/LSTM.json").show("musicxml")